The perceptron algorithm is a type of linear classification algorithm used to classify data into two categories. It is a simple algorithm that learns from the mistakes made during the classification process and adjusts the weights of the input features to improve the accuracy of the classification. 

```python 
y_pred = sign(w0 + w1*x1 + w2*x2 + ... + wn*xn)
wi = wi + learning_rate * (target - y_pred) * xi
```

Here is an implementation of the perceptron algorithm in Python:

In [1]:
import numpy as np

class Perceptron:
    def __init__(self, lr=0.01, n_iter=100):
        self.lr = lr
        self.n_iter = n_iter

    def fit(self, X, y):
        self.weights = np.zeros(1 + X.shape[1])
        self.errors = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.lr * (target - self.predict(xi))
                self.weights[1:] += update * xi
                self.weights[0] += update
                errors += int(update != 0.0)
            self.errors.append(errors)
        return self

    def net_input(self, X):
        return np.dot(X, self.weights[1:]) + self.weights[0]

    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)


The Perceptron class has the following methods:

__init__(self, lr=0.01, n_iter=100): Initializes the perceptron with a learning rate (lr) and number of iterations (n_iter) to perform during training.

fit(self, X, y): Trains the perceptron on the input data X and target labels y. The method initializes the weights to zero and iterates through the data n_iter times, adjusting the weights after each misclassification. The method returns the trained perceptron.

net_input(self, X): Computes the weighted sum of inputs and bias.

predict(self, X): Predicts the class label for a given input X based on the current weights.

To use the perceptron algorithm, you can create an instance of the Perceptron class, and then call the fit method with your input data X and target labels y. Here is an example usage:

In [2]:
X = np.array([[2.0, 1.0], [3.0, 4.0], [4.0, 2.0], [3.0, 1.0]])
y = np.array([-1, 1, 1, -1])
perceptron = Perceptron()
perceptron.fit(X, y)

new_X = np.array([[5.0, 2.0], [1.0, 3.0]])
perceptron.predict(new_X)



array([-1,  1])

In [3]:
import numpy as np

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        """
        Initialize the neural network parameters and hyperparameters.
        
        Args:
        - input_size (int): Number of input features.
        - hidden_size (int): Number of neurons in the hidden layer.
        - output_size (int): Number of output neurons.
        - learning_rate (float): Learning rate for weight updates.
        """
        # Set random seed for reproducibility
        np.random.seed(0)

        # Initialize weights and biases
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)
        self.bias_hidden = np.random.rand(hidden_size)
        self.bias_output = np.random.rand(output_size)
        
        # Set learning rate
        self.learning_rate = learning_rate

    def sigmoid(self, x):
        """
        Compute the sigmoid activation function.
        
        Args:
        - x (np.ndarray): Input values.
        
        Returns:
        - np.ndarray: Sigmoid of the input.
        """
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """
        Compute the derivative of the sigmoid function.
        
        Args:
        - x (np.ndarray): Input values.
        
        Returns:
        - np.ndarray: Derivative of the sigmoid function at the input.
        """
        sig = self.sigmoid(x)
        return sig * (1 - sig)

    def forward(self, X):
        """
        Perform a forward pass through the network.
        
        Args:
        - X (np.ndarray): Input data.
        
        Returns:
        - np.ndarray: Output of the network.
        """
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.y_pred = self.sigmoid(self.final_input)
        return self.y_pred

    def backward(self, X, y):
        """
        Perform a backward pass to update weights and biases.
        
        Args:
        - X (np.ndarray): Input data.
        - y (np.ndarray): True labels.
        """
        # Compute loss
        loss = np.mean((y - self.y_pred) ** 2)

        # Compute gradients
        d_loss_y_pred = self.y_pred - y
        d_y_pred_final_input = self.sigmoid_derivative(self.final_input)
        d_final_input_weights_hidden_output = self.hidden_output

        d_loss_final_input = d_loss_y_pred * d_y_pred_final_input
        d_loss_weights_hidden_output = np.dot(d_final_input_weights_hidden_output.T, d_loss_final_input)
        d_loss_bias_output = np.sum(d_loss_final_input, axis=0)

        d_hidden_output_hidden_input = self.sigmoid_derivative(self.hidden_output)
        d_hidden_input_weights_input_hidden = X

        d_loss_hidden_output = np.dot(d_loss_final_input, self.weights_hidden_output.T)
        d_loss_hidden_input = d_loss_hidden_output * d_hidden_output_hidden_input
        d_loss_weights_input_hidden = np.dot(d_hidden_input_weights_input_hidden.T, d_loss_hidden_input)
        d_loss_bias_hidden = np.sum(d_loss_hidden_input, axis=0)

        # Update weights and biases
        self.weights_hidden_output -= self.learning_rate * d_loss_weights_hidden_output
        self.bias_output -= self.learning_rate * d_loss_bias_output
        self.weights_input_hidden -= self.learning_rate * d_loss_weights_input_hidden
        self.bias_hidden -= self.learning_rate * d_loss_bias_hidden

        return loss

    def train(self, X, y, epochs=10000):
        """
        Train the neural network.
        
        Args:
        - X (np.ndarray): Input data.
        - y (np.ndarray): True labels.
        - epochs (int): Number of training epochs.
        """
        for epoch in range(epochs):
            # Forward pass
            self.forward(X)
            # Backward pass
            loss = self.backward(X, y)
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        """
        Make predictions using the trained model.
        
        Args:
        - X (np.ndarray): Input data.
        
        Returns:
        - np.ndarray: Predicted values.
        """
        return self.forward(X)

# Example usage
if __name__ == "__main__":
    # Define the neural network
    input_size = 2
    hidden_size = 2
    output_size = 1
    learning_rate = 0.1
    nn = SimpleNeuralNetwork(input_size, hidden_size, output_size, learning_rate)

    # Training data for XOR problem
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y = np.array([[0], [1], [1], [0]])

    # Train the network
    nn.train(X, y, epochs=10000)

    # Make predictions
    predictions = nn.predict(X)
    print("Predictions:")
    print(predictions)


Epoch 0, Loss: 0.3764518162273597
Epoch 1000, Loss: 0.24965358535236715
Epoch 2000, Loss: 0.2490915145023216
Epoch 3000, Loss: 0.24868501067882615
Epoch 4000, Loss: 0.2485159855419558
Epoch 5000, Loss: 0.24860723024344678
Epoch 6000, Loss: 0.24887270591470728
Epoch 7000, Loss: 0.24917823508506215
Epoch 8000, Loss: 0.24942796667025774
Epoch 9000, Loss: 0.24958802454526538
Predictions:
[[0.48096412]
 [0.50103142]
 [0.50091049]
 [0.51890476]]


In [6]:
k=SimpleNeuralNetwork(5,10,1)

In [8]:
k.weights_hidden_output

array([[0.57019677],
       [0.43860151],
       [0.98837384],
       [0.10204481],
       [0.20887676],
       [0.16130952],
       [0.65310833],
       [0.2532916 ],
       [0.46631077],
       [0.24442559]])

In [21]:
k.weights_input_hidden.shape


(5, 10)

In [20]:
np.random.rand(3,5)

array([[0.4686512 , 0.97676109, 0.60484552, 0.73926358, 0.03918779],
       [0.28280696, 0.12019656, 0.2961402 , 0.11872772, 0.31798318],
       [0.41426299, 0.0641475 , 0.69247212, 0.56660145, 0.26538949]])

In [23]:
k.bias_output

array([0.97645947])

In [227]:
import numpy as np

class NeuralNet:
    
    def __init__(self, input_dim, hid_dim, output_dim, lr=0.001):
        self.input_hidden_weights = np.random.randn(input_dim, hid_dim)
        self.hidden_output_weights = np.random.randn(hid_dim, output_dim)
        self.hidden_bias = np.random.rand(hid_dim)
        self.output_bias = np.random.rand(output_dim)
        self.lr = lr
     
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def derivative_sigmoid(self, x):
        sig = self.sigmoid(x)
        return sig * (1 - sig)
    
    def forward(self, X):
        self.hidden_input = np.dot(X, self.input_hidden_weights) + self.hidden_bias
        self.hidden_output = np.dot(self.hidden_input, self.hidden_output_weights) + self.output_bias
        self.output_final = self.sigmoid(self.hidden_output)
        self.y_val = np.round(self.output_final, 3)
        return self.y_val
    
    def train(self, X, y, epochs):
        for _ in range(epochs):
            self.forward(X)
            self.backward(X, y)
    
    def backward(self, X, y):
        m = X.shape[0]  # Number of data points

        # Calculate loss
        self.loss = np.mean((y - self.y_val) ** 2)
        
        # Output layer error
        dl_dy_val = self.y_val - y
        dl_dz_output = dl_dy_val * self.derivative_sigmoid(self.hidden_output)
        
        # Gradients for hidden-output weights and biases
        dl_hidden_output_weights = np.dot(self.hidden_input.T, dl_dz_output) / m
        dl_output_bias = np.sum(dl_dz_output, axis=0) / m
        
        # Hidden layer error
        dl_hidden_output = np.dot(dl_dz_output, self.hidden_output_weights.T)
        
        # Gradients for input-hidden weights and biases
        dl_input_hidden_weights = np.dot(X.T, dl_hidden_output) / m
        dl_hidden_bias = np.sum(dl_hidden_output, axis=0) / m
        
        # Update weights and biases
        self.hidden_output_weights -= self.lr * dl_hidden_output_weights
        self.output_bias -= self.lr * dl_output_bias
        self.input_hidden_weights -= self.lr * dl_input_hidden_weights
        self.hidden_bias -= self.lr * dl_hidden_bias


In [219]:
a1=NeuralNet(4,10,1)

In [220]:
X=np.array([[1,2,3,4],[1,4,2,1]],dtype=np.float32)

In [221]:
a1.input_hidden_weights.shape

(4, 10)

In [222]:
X.shape

(2, 4)

In [223]:
a1.input_hidden_weights.shape

(4, 10)

In [224]:
a1.forward(X)

array([[0.687],
       [0.532]])

In [225]:
a1.train(X,np.array([[1],[4]]),10)

In [226]:
a1.output_final

array([[0.6901636 ],
       [0.53816513]])

In [176]:
k1=np.random.randn(4,5)

In [177]:
X=np.array([[1,2,3,4],[1,4,2,1]],dtype=np.float32)

In [178]:
np.dot(X,k1)

array([[-0.21869238, -4.15386666,  8.59065342,  0.11165262,  7.52637846],
       [-0.53758915,  0.47764507,  5.96139666, -0.91113334,  4.64987846]])

In [179]:
w1=np.array([1,2,3])

In [180]:
x1=np.array([[1,2,3],
             [4,5,6],
             [7,8,9],
             [10,11,12]
            ])

In [181]:
np.dot(x1,w1)

array([14, 32, 50, 68])

In [160]:
np.matmul(w1,x1)

array([70, 80, 90])